# Custom User-based Model
The present notebooks aims at creating a UserBased class that inherits from the Algobase class (surprise package) and that can be customized with various similarity metrics, peer groups and score aggregation functions. 

In [2]:
# reloads modules automatically before entering the execution of code
%load_ext autoreload
%autoreload 2

# standard library imports
# -- add new imports here --

# third parties imports
import numpy as np 
import pandas as pd
from surprise import AlgoBase
# -- add new imports here --

# local imports
from constants import Constant as C
from loaders import load_ratings
# -- add new imports here --
from surprise import Dataset, KNNWithMeans
import matplotlib.pyplot as plt
import seaborn as sns

###
import heapq


        userId  movieId  rating   timestamp
0            1       31     2.5  1260759144
1            1     1029     3.0  1260759179
2            1     1061     3.0  1260759182
3            1     1129     2.0  1260759185
4            1     1172     4.0  1260759205
...        ...      ...     ...         ...
99999      671     6268     2.5  1065579370
100000     671     6269     4.0  1065149201
100001     671     6365     4.0  1070940363
100002     671     6385     2.5  1070979663
100003     671     6565     3.5  1074784724

[100004 rows x 4 columns]


# 1. Loading Data
Prepare a dataset in order to help implementing a user-based recommender system

In [3]:
# -- load data, build trainset and anti testset --
ratings = load_ratings(surprise_format=True)
trainset = ratings.build_full_trainset()
anti_testset = trainset.build_anti_testset()

# 2. Explore Surprise's user-based algorithm
Displays user-based predictions and similarity matrix on the test dataset using the KNNWithMeans class

In [5]:
# -- using surprise's user-based algorithm, explore the impact of different parameters and displays predictions --
sim_options = {
    'name': 'msd',  
    'user_based': True,  
    'min_support': 3  
}

algo = KNNWithMeans(k=3, min_k=2, sim_options=sim_options)
algo.fit(trainset)
prediction = algo.predict(uid=11, iid=364)
print(f"Prédiction pour l'utilisateur 11 et l'élément 364 : {prediction.est}")


predictions = algo.test(anti_testset)
for pred in predictions[:30]:
    print(f"Utilisateur {pred.uid} a évalué l'élément {pred.iid} avec une note de {pred.est:.2f} (actual_k = {pred.details.get('actual_k', 'N/A')})")

    
# Similarity matrix 
print("\n--- Aperçu de la matrice de similarité utilisateur-utilisateur ---")
sim_matrix = algo.sim  
n_max = min(10, sim_matrix.shape[0])  
for i in range(n_max):
    print(f"Similarités de l'utilisateur interne {i} avec les autres : {sim_matrix[i, :n_max]}")

# Stockage de la matrice de similarité dans un fichier CSV
sim_df = pd.DataFrame(sim_matrix)
sim_df.to_csv("similarity_matrix.csv", index=False)
print("\nLa matrice de similarité a été sauvegardée dans 'similarity_matrix.csv'.")

Computing the msd similarity matrix...
Done computing similarity matrix.
Prédiction pour l'utilisateur 11 et l'élément 364 : 4.252920516369196
Utilisateur 1 a évalué l'élément 10 avec une note de 2.62 (actual_k = 3)
Utilisateur 1 a évalué l'élément 17 avec une note de 3.19 (actual_k = 3)
Utilisateur 1 a évalué l'élément 39 avec une note de 2.53 (actual_k = 3)
Utilisateur 1 a évalué l'élément 47 avec une note de 2.98 (actual_k = 3)
Utilisateur 1 a évalué l'élément 50 avec une note de 4.04 (actual_k = 3)
Utilisateur 1 a évalué l'élément 52 avec une note de 2.13 (actual_k = 3)
Utilisateur 1 a évalué l'élément 62 avec une note de 2.77 (actual_k = 3)
Utilisateur 1 a évalué l'élément 110 avec une note de 2.70 (actual_k = 3)
Utilisateur 1 a évalué l'élément 144 avec une note de 2.13 (actual_k = 3)
Utilisateur 1 a évalué l'élément 150 avec une note de 1.80 (actual_k = 3)
Utilisateur 1 a évalué l'élément 153 avec une note de 2.13 (actual_k = 3)
Utilisateur 1 a évalué l'élément 161 avec une note

observations minK: 

- observations minK 2 par rapport à 1 sont similaire ou légerement plus elevé :Cela s'explique par le fait que l’algorithme impose une faible  contraintes sur le nombre minimum de voisins requis pour effectuer une prédiction. Avec des valeurs faibles de min_k, comme 1 ou 2, les prédictions peuvent être faites même avec très peu de voisins ce qui explique la faible variation entre les deux cas.
Cependant, pour certaines prédictions où le nombre de voisins disponibles est inférieur à 2, l’algorithme utilise une valeur par défaut.

- observation mink= 3: on augmente le nombre minimum de voisins requis pour faire une prédiction ce qui rend l’algorithme plus strict.
Dans les 30 premières prédictions observées la majorité des valeurs restent similaires mais plusieurs prédictions augmentent, notamment celle de l'utilisateur 11 pour l’élément 364. Cette hausse est due au fait qu’il n’y avait pas assez de voisins (moins de 3) pour faire une prédiction personnalisée. L’algorithme a donc utilisé une valeur par défaut par exemple  ce qui peut expliquer l’augmentation.

observations min support: 

- Plus la valeur de min_support est élevée, plus actual_k a tendance à diminuer.Cela s'explique par le fait qu’un min_support plus grand impose un critère plus strict : chaque voisin doit avoir au moins ce nombre d’évaluations communes avec l’utilisateur cible (par exemple, min_support = 3 signifie qu'il doit avoir au moins 3 évaluations communes avec l'utilisateur cible pour être pris en compte dans le calcul).Ainsi, moins de voisins respectent cette condition  ce qui réduit le nombre de voisins utilisables pour la prédiction et donc reduit le actual_k. 

- La variable actual_k représente le nombre de voisins qui ont réellement contribué à la prédiction.Plus actual_k est élevé, plus la prédiction est considérée comme fiable car elle repose sur un plus grand nombre d’avis pertinents.

# 3. Implement and explore a customizable user-based algorithm
Create a self-made user-based algorithm allowing to customize the similarity metric, peer group calculation and aggregation function

In [7]:
class UserBased(AlgoBase):
    def __init__(self, k=3, min_k=1, sim_options={}, **kwargs):
        AlgoBase.__init__(self, sim_options=sim_options, **kwargs)
        self.k = k
        self.min_k = min_k

        
    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        
        self.compute_rating_matrix()
        
        self.compute_similarity_matrix()
        
        self.mean_ratings = []
        for u in range(self.trainset.n_users):
            user_ratings = []
            for (_, rating) in self.trainset.ur[u]: 
                user_ratings.append(rating)
            if user_ratings:
                mean_rating = np.mean(user_ratings)
            else:
                mean_rating = float('nan')  
            self.mean_ratings.append(mean_rating)

    
    def estimate(self, u, i):
            if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
                raise np.nan 
            
            estimate = self.mean_ratings[u]

            
            neighbors = []
            for (v, rating) in self.trainset.ir[i]:  
                if v == u:
                    continue  

                sim_uv = self.sim[u, v] 

                if sim_uv > 0 and not np.isnan(self.ratings_matrix[v, i]): 
                    mean_v = self.mean_ratings[v]  
                    neighbors.append((sim_uv, rating - mean_v))

            
            top_k_neighbors = heapq.nlargest(self.k, neighbors, key=lambda x: x[0])

            
            actual_k = 0
            weighted_sum = 0.0
            sum_sim = 0.0

            for sim, rating_diff in top_k_neighbors:
                if actual_k == self.k:
                    break
                weighted_sum += sim * rating_diff
                sum_sim += sim
                actual_k += 1

            # Check
            if actual_k >= self.min_k and sum_sim > 0:
                estimate += weighted_sum / sum_sim

            return estimate


                            
    def compute_rating_matrix(self):
        # -- implement here the compute_rating_matrix function --
        self.ratings_matrix = np.empty((self.trainset.n_users, self.trainset.n_items))
        self.ratings_matrix[:] = np.nan
        for u in range(self.trainset.n_users): 
            for i, rating in self.trainset.ur[u]:
                self.ratings_matrix[u, i] = rating

    
    def compute_similarity_matrix(self):
        m = self.trainset.n_users
        ratings_matrix = self.ratings_matrix
        min_support = self.sim_options.get('min_support', 1)
        sim_name = self.sim_options.get("name", "msd") 

        # Similarity matrix
        self.sim = np.eye(m)

        for i in range(m):
            for j in range(i + 1, m):  
                row_i = ratings_matrix[i]
                row_j = ratings_matrix[j]

                if sim_name == "jaccard":
                    sim = self.jaccard_similarity(row_i, row_j)
                    support = np.sum(~np.isnan(row_i) & ~np.isnan(row_j))
                elif sim_name == "msd":
                    diff = row_i - row_j
                    support = np.sum(~np.isnan(diff))
                    if support >= min_support:
                        msd = np.nanmean((diff[~np.isnan(diff)]) ** 2)
                        sim = 1 / (1 + msd)
                    else:
                        sim = 0
                else:
                    
                    diff = row_i - row_j
                    support = np.sum(~np.isnan(diff))
                    if support >= min_support:
                        msd = np.nanmean((diff[~np.isnan(diff)]) ** 2)
                        sim = 1 / (1 + msd)
                    else:
                        sim = 0

                if support >= min_support:
                    self.sim[i, j] = sim
                    self.sim[j, i] = sim

    def jaccard_similarity(self, row_i, row_j):
        
        mask_i = ~np.isnan(row_i)
        mask_j = ~np.isnan(row_j)

        intersection = np.sum(mask_i & mask_j)
        union = np.sum(mask_i | mask_j)

        if union == 0:
            return 0.0
        return intersection / union


# 4. Compare KNNWithMeans with UserBased
Try to replicate KNNWithMeans with your self-made UserBased and check that outcomes are identical

In [8]:
# -- assert that predictions are the same with different sim_options --
#comparaison entre requltat KNNwithKmeans et resultqts user based 
import surprise
from surprise import accuracy

sim_options = {
    'name': 'msd',
    'user_based': True,
    'min_support': 3
}
k = 3
min_k = 2


userbased_algo = UserBased(k=k, min_k=min_k, sim_options=sim_options)
userbased_algo.fit(trainset)


knn_algo = KNNWithMeans(k=k, min_k=min_k, sim_options=sim_options)
knn_algo.fit(trainset)


anti_testset = trainset.build_anti_testset()


print(f"{'UID':>5} {'IID':>5} {'KNN est.':>10} {'UserBased est.':>15} {'Diff':>10}")
print("-" * 50)

for i, (uid, iid, _) in enumerate(anti_testset[:30]):
    pred_knn = knn_algo.predict(uid, iid)
    pred_userbased = userbased_algo.predict(uid, iid)

    diff = abs(pred_knn.est - pred_userbased.est)

    print(f"{uid:>5} {iid:>5} {pred_knn.est:10.4f} {pred_userbased.est:15.4f} {diff:10.4f}")



Computing the msd similarity matrix...
Done computing similarity matrix.
  UID   IID   KNN est.  UserBased est.       Diff
--------------------------------------------------
    1    10     2.6248          2.6248     0.0000
    1    17     3.1901          3.1901     0.0000
    1    39     2.5295          2.5295     0.0000
    1    47     2.9796          2.9796     0.0000
    1    50     4.0432          4.0432     0.0000
    1    52     2.1271          2.1271     0.0000
    1    62     2.7740          2.7740     0.0000
    1   110     2.6954          2.6954     0.0000
    1   144     2.1305          2.1305     0.0000
    1   150     1.8002          1.8002     0.0000
    1   153     2.1304          2.1304     0.0000
    1   161     2.6733          2.6733     0.0000
    1   165     3.0645          3.0645     0.0000
    1   168     2.8552          2.8552     0.0000
    1   185     2.4117          2.4117     0.0000
    1   186     1.5386          1.5386     0.0000
    1   208     1.2282    

# 5. Compare MSD and Jacard
Compare predictions made with MSD similarity and Jacard similarity


In [9]:
import surprise
from surprise import accuracy

sim_options_userbased = {
    'name': 'jaccard',
    'user_based': True,
    'min_support': 3
}

sim_options_userbased2 = {
    'name': 'msd',
    'user_based': True,
    'min_support': 3
}
k = 3
min_k = 2


userbased_algo = UserBased(k=k, min_k=min_k, sim_options=sim_options_userbased)
userbased_algo.fit(trainset)


userbased_algo2 = UserBased(k=k, min_k=min_k, sim_options=sim_options_userbased2)
userbased_algo2.fit(trainset)


anti_testset = trainset.build_anti_testset()

print(f"{'UID':>5} {'IID':>5} {'userbased_algo2 est.':>10} {'UserBased est.':>15} {'Diff':>10}")
print("-" * 50)

for i, (uid, iid, _) in enumerate(anti_testset[:30]):
    pred_userbased_algo2 = userbased_algo2.predict(uid, iid)
    pred_userbased = userbased_algo.predict(uid, iid)

    diff = abs(pred_userbased_algo2.est - pred_userbased.est)

    print(f"{uid:>5} {iid:>5} {pred_userbased_algo2.est:10.4f} {pred_userbased.est:15.4f} {diff:10.4f}")


  UID   IID userbased_algo2 est.  UserBased est.       Diff
--------------------------------------------------
    1    10     2.6248          2.4304     0.1944
    1    17     3.1901          3.3088     0.1187
    1    39     2.5295          2.0776     0.4519
    1    47     2.9796          2.9232     0.0565
    1    50     4.0432          3.0949     0.9483
    1    52     2.1271          2.2279     0.1007
    1    62     2.7740          2.5136     0.2604
    1   110     2.6954          3.2296     0.5342
    1   144     2.1305          2.9085     0.7780
    1   150     1.8002          2.0321     0.2319
    1   153     2.1304          1.3591     0.7713
    1   161     2.6733          2.5791     0.0942
    1   165     3.0645          2.2383     0.8262
    1   168     2.8552          2.0951     0.7602
    1   185     2.4117          2.1663     0.2454
    1   186     1.5386          1.8044     0.2658
    1   208     1.2282          1.3135     0.0853
    1   222     2.6069          2.6278 